In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import re

import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv("./data/d_cll.csv")
data.head()

,preserves,work_order_number,site_number,site_type,date_created,month_year,date_completed,provider_name,wo_current_status,asset_category,...,wo_priority,manufacturer,asset_in_service_date,level_1_organization_name,site_zipcode,sales_sq_ft,asset_barcode,asset_type,asset_status,count_date
0,Frozen Food,12904684,A0892,SUPERMARKET,2012-01-02 03:05:00,2012-01,2012-01-02 10:00:00,Space Engineering services LTD,EOL,REFRIGERATION,...,Standard SLA,CARTER,2011-01-06 15:19:00,Region N10,DN31 1UF,42639.0,SSL00567391,HALF GLASS DOOR & WELL,In Service,2684
1,Frozen Food,12904685,A0892,SUPERMARKET,2012-01-02 03:05:00,2012-01,2012-01-02 09:59:00,Space Engineering services LTD,EOL,REFRIGERATION,...,Standard SLA,CARTER,2011-01-06 15:19:00,Region N10,DN31 1UF,42639.0,SSL00567398,FULL GLASS DOOR,In Service,2684
2,Frozen Food,12904853,A0892,SUPERMARKET,2012-01-02 05:10:00,2012-01,2012-01-02 10:01:00,Space Engineering services LTD,EOL,REFRIGERATION,...,Standard SLA,CARTER,2011-01-06 15:19:00,Region N10,DN31 1UF,42639.0,SSL00567399,FULL GLASS DOOR,In Service,2684
3,"Meat, Fish and Poultry",12927899,A0416,SUPERMARKET,2012-01-02 08:40:00,2012-01,2012-02-02 13:03:00,CMS Solutions Limited,EOL,REFRIGERATION,...,Standard SLA,LINDE,2002-01-08 05:00:00,Region S01,BR4 0PU,13423.0,SSL00225160,REMOTE MULTIDECK,In Service,2684
4,Frozen Food,12932231,A0003,SUPERMARKET,2012-01-02 13:33:00,2012-01,2012-08-02 19:39:00,CMS Solutions Limited,EOL,REFRIGERATION,...,Standard SLA,RADFORD RETAIL SYSTEMS,2009-03-15 04:00:00,Region S02,DA11 8JH,37870.0,SSL00583503,FULL GLASS DOOR,Scrapped,2684


In [3]:
%%time
data["date_created"] = pd.DatetimeIndex(data.date_created)
data["date_completed"] = pd.DatetimeIndex(data.date_completed)
data["asset_in_service_date"] = pd.DatetimeIndex(data.asset_in_service_date)

CPU times: total: 109 ms
Wall time: 973 ms


In [4]:
data_prueba = data.date_created.dt.to_period('M').value_counts().reset_index()
data_prueba.columns = ["month_year","count_date"]
data_prueba = data_prueba.sort_values(by=['month_year'])
data_prueba.head()

,month_year,count_date
47,2012-01,2684
44,2012-02,4783
40,2012-03,6833
43,2012-04,5467
14,2012-05,12312


In [6]:
target = data["count_date"]
data_filtered = data[["count_date","site_type","date_created","date_completed","provider_name","asset_category","problem_type_name",
               "requested_by","wo_priority","manufacturer","asset_in_service_date","sales_sq_ft",
               "asset_type","asset_status","preserves"]]
data_filtered.head()

,count_date,site_type,date_created,date_completed,provider_name,asset_category,problem_type_name,requested_by,wo_priority,manufacturer,asset_in_service_date,sales_sq_ft,asset_type,asset_status,preserves
0,2684,SUPERMARKET,2012-01-02 03:05:00,2012-01-02 10:00:00,Space Engineering services LTD,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CARTER,2011-01-06 15:19:00,42639.0,HALF GLASS DOOR & WELL,In Service,Frozen Food
1,2684,SUPERMARKET,2012-01-02 03:05:00,2012-01-02 09:59:00,Space Engineering services LTD,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CARTER,2011-01-06 15:19:00,42639.0,FULL GLASS DOOR,In Service,Frozen Food
2,2684,SUPERMARKET,2012-01-02 05:10:00,2012-01-02 10:01:00,Space Engineering services LTD,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CARTER,2011-01-06 15:19:00,42639.0,FULL GLASS DOOR,In Service,Frozen Food
3,2684,SUPERMARKET,2012-01-02 08:40:00,2012-02-02 13:03:00,CMS Solutions Limited,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,LINDE,2002-01-08 05:00:00,13423.0,REMOTE MULTIDECK,In Service,"Meat, Fish and Poultry"
4,2684,SUPERMARKET,2012-01-02 13:33:00,2012-08-02 19:39:00,CMS Solutions Limited,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,RADFORD RETAIL SYSTEMS,2009-03-15 04:00:00,37870.0,FULL GLASS DOOR,Scrapped,Frozen Food


In [7]:
cat_columns = data_filtered.select_dtypes(['object']).columns
data_filtered[cat_columns] = data_filtered[cat_columns].apply(lambda x: pd.factorize(x)[0])
data_filtered.head()

,count_date,site_type,date_created,date_completed,provider_name,asset_category,problem_type_name,requested_by,wo_priority,manufacturer,asset_in_service_date,sales_sq_ft,asset_type,asset_status,preserves
0,2684,0,2012-01-02 03:05:00,2012-01-02 10:00:00,0,0,0,0,0,0,2011-01-06 15:19:00,42639.0,0,0,0
1,2684,0,2012-01-02 03:05:00,2012-01-02 09:59:00,0,0,0,0,0,0,2011-01-06 15:19:00,42639.0,1,0,0
2,2684,0,2012-01-02 05:10:00,2012-01-02 10:01:00,0,0,0,0,0,0,2011-01-06 15:19:00,42639.0,1,0,0
3,2684,0,2012-01-02 08:40:00,2012-02-02 13:03:00,1,0,0,0,0,1,2002-01-08 05:00:00,13423.0,2,0,1
4,2684,0,2012-01-02 13:33:00,2012-08-02 19:39:00,1,0,0,0,0,2,2009-03-15 04:00:00,37870.0,1,1,2


In [8]:
data_filtered["date_created_day"] = data_filtered["date_created"].apply(lambda x: x.day)
data_filtered["date_completed_day"] = data_filtered["date_completed"].apply(lambda x: x.day)
data_filtered["asset_in_service_date_day"] = data_filtered["asset_in_service_date"].apply(lambda x: x.day)

In [9]:
data_filtered["date_created_month"] = data_filtered["date_created"].apply(lambda x: x.month)
data_filtered["date_completed_month"] = data_filtered["date_completed"].apply(lambda x: x.month)
data_filtered["asset_in_service_date_month"] = data_filtered["asset_in_service_date"].apply(lambda x: x.month)

In [10]:
data_filtered["date_created_year"] = data_filtered["date_created"].apply(lambda x: x.year)
data_filtered["date_completed_year"] = data_filtered["date_completed"].apply(lambda x: x.year)
data_filtered["asset_in_service_date_year"] = data_filtered["asset_in_service_date"].apply(lambda x: x.year)

In [11]:
data_filtered["date_created_hour"] = data_filtered["date_created"].apply(lambda x: x.hour)
data_filtered["date_completed_hour"] = data_filtered["date_completed"].apply(lambda x: x.hour)
data_filtered["asset_in_service_date_hour"] = data_filtered["asset_in_service_date"].apply(lambda x: x.hour)

In [12]:
data_filtered.drop("date_created",axis=1,inplace=True)
data_filtered.drop("date_completed",axis=1,inplace=True)
data_filtered.drop("asset_in_service_date",axis=1,inplace=True)

In [13]:
data_filtered.head()

,count_date,site_type,provider_name,asset_category,problem_type_name,requested_by,wo_priority,manufacturer,sales_sq_ft,asset_type,...,asset_in_service_date_day,date_created_month,date_completed_month,asset_in_service_date_month,date_created_year,date_completed_year,asset_in_service_date_year,date_created_hour,date_completed_hour,asset_in_service_date_hour
0,2684,0,0,0,0,0,0,0,42639.0,0,...,6,1,1,1,2012,2012,2011,3,10,15
1,2684,0,0,0,0,0,0,0,42639.0,1,...,6,1,1,1,2012,2012,2011,3,9,15
2,2684,0,0,0,0,0,0,0,42639.0,1,...,6,1,1,1,2012,2012,2011,5,10,15
3,2684,0,1,0,0,0,0,1,13423.0,2,...,8,1,2,1,2012,2012,2002,8,13,5
4,2684,0,1,0,0,0,0,2,37870.0,1,...,15,1,8,3,2012,2012,2009,13,19,4


In [16]:
data_filtered.shape

(527644, 24)

In [15]:
data_3points = data_filtered[data_filtered.date_created_year != 2015]
data_3points.shape # alarmas que no son del 2015

(403928, 24)

In [74]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [75]:
X = data_3points[data_3points.columns.tolist()[1:]].to_numpy()
y = data_3points[["count_date"]].to_numpy()

In [76]:
scaler=StandardScaler()
X=scaler.fit_transform(X)

In [77]:
model = linear_model.LinearRegression()
model.fit(X,y)

LinearRegression()

In [78]:
data2015 = data_filtered[data_filtered.date_created_year == 2015]

In [79]:
X_test = data2015[data2015.columns.tolist()[1:]].to_numpy()
y_test = data2015[["count_date"]].to_numpy()

In [80]:
X_test = scaler.transform(X_test)

In [ ]:
y_pred = model.predict(X_test)

In [86]:
y_pred = np.int32(y_pred)
y_t = np.reshape(y_test,len(y_test))
y_p = np.reshape(y_pred,len(y_pred))

In [87]:
dicte={"y_test":y_t,"y_pred":y_p}
pd.DataFrame(dicte)

,y_test,y_pred
0,13835,13512
1,13835,13958
2,13835,12521
3,13835,12550
4,13835,12239
...,...,...
123711,13835,14903
123712,13835,14184
123713,13835,14859
123714,13835,14698


In [88]:
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
print('Variance score: %.2f' % r2_score(y_test, y_pred))

Mean squared error: 33750225.83
Variance score: -3.97
